### Tutors - expected math exam results
__Predict average math exam results for students of the tutors__  
https://www.kaggle.com/c/tutors-expected-math-exam-results

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
TRAIN_DATASET_PATH = './datasets/train.csv'
TEST_DADASET_PATH = './datasets/test.csv'

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.info()

In [ ]:
train_df.describe().T

In [ ]:
# Compute the correlation matrix
df_corr = train_df.corr()

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(150, 275, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df_corr['mean_exam_points']

In [ ]:
train_df.hist(figsize =(12,14),bins = 15, grid = True)
plt.show()

In [27]:
class DecisionTreeRegressor:
    
    class Node:
    
        def __init__(self, index, t, true_branch, false_branch):
            self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
            self.t = t  # значение порога
            self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
            self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        

    class Leaf:

        def __init__(self, data, target_values):
            self.data = data
            self.target_values = target_values
            self.prediction = self.predict()

        def predict(self):
            return self.target_values.mean()
        
    def _H(self, target_values):
        return sum(target_values - target_values.mean()) / len(target_values)

    def _quality(self, left_target_values, right_target_values, current_H):
        p = float(left_target_values.shape[0]) / (left_target_values.shape[0] + right_target_values.shape[0])
        return current_H - p * self._H(left_target_values) - (1 - p) * self._H(right_target_values)

    def _split(self, data, target_values, index, t):

        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_target_values = target_values[left]
        false_target_values = target_values[right]

        return true_data, false_data, true_target_values, false_target_values

    def _find_best_split(self, data, target_values):

        #  обозначим минимальное количество объектов в узле
        min_leaf = 5

        current_H = self._H(target_values)

        best_quality = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_target_values, false_target_values = self._split(data, target_values, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue

                current_quality = self._quality(true_target_values, false_target_values, current_H)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index

    def _build_tree(self, data, target_values, current_depth):
        
        # Если параметр current_deep не None уменьшаю счетчик оставшейся глубины дерева
        # Как он достигает 0, прекращаю цикл
        if current_depth is not None:
            if current_depth == 0:
                return self.Leaf(data, target_values)
            current_depth -= 1
        
        quality, t, index = self._find_best_split(data, target_values)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return self.Leaf(data, target_values)

        true_data, false_data, true_target_values, false_target_values = self._split(data, target_values, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self._build_tree(true_data, true_target_values, current_depth)
        false_branch = self._build_tree(false_data, false_target_values, current_depth)
    
        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return self.Node(index, t, true_branch, false_branch)
        
    def build(self, data, target_values, max_depth=None):
        """max_deep - параметр, ограничивающий максимальную глубину создаваемого дерева. 
        Если None, то глубина дерева не ограничена"""
        self.tree = self._build_tree(data, target_values, max_depth)

    def _classify_object(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, self.Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self._classify_object(obj, node.true_branch)
        else:
            return self._classify_object(obj, node.false_branch)

    def predict(self, data):

        classes = []
        for obj in data:
            prediction = self._classify_object(obj, self.tree)
            classes.append(prediction)
        return classes

In [36]:
class GradientBoosting:
    
    @staticmethod
    def mean_squared_error(y_real, prediction):
        return (sum((y_real - prediction)**2)) / len(y_real)
    
    @staticmethod
    def bias(y, z):
        return 2 * (y - z)
    
    def fit(self, n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    
        # Деревья будем записывать в список
        trees = []

        for i in range(n_trees):
            tree = DecisionTreeRegressor()

            # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
            # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
            if len(trees) == 0:
                # обучаем первое дерево на обучающей выборке
                tree.build(X_train, y_train, max_depth=max_depth)

            else:
                # Получим ответы на текущей композиции
                target = self.predict(X_train, trees, coefs, eta)

                # алгоритмы начиная со второго обучаем на сдвиг
                tree.build(X_train, self.bias(y_train, target), max_depth=max_depth)

            trees.append(tree)
            
        return trees, train_errors, test_errors
        
    def predict(self, X, trees_list, coef_list, eta):
        # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
        # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
        return np.array([sum([eta* coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

In [37]:
def r_squared_score(y_true, y_pred):
        corr_matrix = np.corrcoef(y_true, y_pred)
        corr = corr_matrix[0,1]
        return corr**2

In [38]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [39]:
# Число деревьев в ансамбле
n_trees = 100

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 5

# Шаг
eta = 0.001

gb_model = GradientBoosting()
trees, train_errors, test_errors = gb_model.fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)

In [40]:
y_pred = gb_model.predict(X_test, trees, coefs, eta)

In [41]:
r_squared_score(y_test, y_pred)

0.39882413459855004